In [ ]:
import numpy as np
from scipy.sparse.linalg import lgmres
from numpy import linalg as la
from scipy.sparse.linalg import LinearOperator
import matplotlib.pyplot as plt

In [ ]:
def Arnoldi(A, v, m):
    w=[]
    V = []
    V.append(np.array(v[:]))
    h=np.zeros((m + 1, m))
    
    for j in range(m):
        w.append(A.dot(V[j]))
                
        for i in range(j+1):
            h[i][j]=np.inner(w[j],V[i])
            w[j]=w[j]-h[i][j]*V[i]
                        
        h[j+1][j]=la.norm(w[j])
                
        if h[j+1][j]==0:
            return np.array(V).T, h[0:-1]
        
        V.append(w[j]/h[j+1][j])
                
    return np.array(V[0:-1]).T, h[0:-1]

In [ ]:
def Linear_Arnoldi(A, v, m):
    w=[]
    V = []
    V.append(np.array(v[:]))
    h=np.zeros((m + 1, m))
    
    for j in range(m):
        w.append(A.dot(V[j]))
                
        for i in range(j+1):
            h[i][j]=np.inner(w[j],V[i])
            w[j]=w[j]-h[i][j]*V[i]
                        
        h[j+1][j]=la.norm(w[j])
                
        if h[j+1][j]==0:
            return np.array(V).T, h[0:-1]
        
        V.append(w[j]/h[j+1][j])           
    return np.array(V[0:-1]).T, h[0:-1]

In [ ]:
def mymatvec(z):
    res = np.zeros((z.size, 1))
    res[0] = (-2-alpha)*z[0]+z[1]
    res[res.size-1] = (-2-alpha)*z[z.size-1]+z[z.size-2]
    for i in range(1,z.size-1):
        res[i] = (-2-alpha)*z[i]+z[i-1]+z[i+1]
    return res

In [ ]:
def accurate_vectors(A,v0,m):
    n=np.zeros((m,m))
    n[0,:]=v0
    for i in range (m-1):
        tmp=A.dot(n[i,:])
        n[i+1,:]=tmp/la.norm(tmp)
    return n

In [ ]:
def accurate_vectors_lgmres(v0,m):
    dim = len(v0)
    A = LinearOperator((dim, dim), matvec=mymatvec)
    Ainv = LinearOperator((dim,dim), lambda v: lgmres(A, v, tol=10**-10, atol=10**-10)[0])
    n=np.zeros((m,dim))
    n[0,:]=v0
    for i in range (m-1):
        
        tmp=Ainv.dot(n[i,:])
        n[i+1,:]=tmp/la.norm(tmp)
    return n

In [ ]:
def approximate_vectors(V,H1,v0,m):
    dim = len(v0)
    n2=np.zeros((m,dim))
    n2[0,:]=v0
    for i in range(m-1):
        tmp=V.dot(H1.dot(V.T.dot(n2[i,:])))
        n2[i+1,:]=tmp/la.norm(tmp)
    return n2

In [ ]:
def relative_error(V,H1,n,m):
    print("Без пересчета")
    r=[]
    vect = V[:,0]
    for i in range(0, m-1):
        vect = V.dot(H1.dot(V.T.dot(vect)))
        vect = vect/np.linalg.norm(vect)
        r.append(np.linalg.norm(n[i+1]-vect)/np.linalg.norm(n[i+1]))
    return r

In [ ]:
def relative_error_Arnoldi(V, H1, n,m,j, tmp):
    print("Пересчет каждый "+str(j)+" раз")
    r=[]
    for i in range(0, m-1):
        if (j-1==0) or i%(j-1)==0:
            V,H=Linear_Arnoldi(A, tmp, m)
            H1=la.inv(H)
        vect = V.dot(H1.dot(V.T.dot(tmp)))
        vect2= lgmres(A,tmp)[0]
        vect = vect/np.linalg.norm(vect)
        tmp = vect
        r.append(np.linalg.norm(n[i+1]-vect)/np.linalg.norm(n[i+1]))
    return r

In [ ]:
alpha=1
m=60
vect_cnt = 30
tmp = np.ones(m)
tmp = tmp/np.linalg.norm(tmp)

In [ ]:

A = LinearOperator((m, m), matvec=mymatvec)
v0=tmp
V,H=Linear_Arnoldi(A,v0,vect_cnt)
H1=la.inv(H)
v=accurate_vectors_lgmres(V[:,0],vect_cnt)
v2=approximate_vectors(V,H1,V[:,0],vect_cnt)
print(v)
print(v2)
r0=relative_error(V,H1,v,vect_cnt)
print(r0)
r1=relative_error_Arnoldi(V, H1, v,vect_cnt,1,tmp)
print(r1)
r2=relative_error_Arnoldi(V,H1,v,vect_cnt,3,tmp)
print(r2)
r3=relative_error_Arnoldi(V,H1,v,vect_cnt,5,tmp)
print(r3)
r4=relative_error_Arnoldi(V,H1,v,vect_cnt,10,tmp)
print(r4)

In [ ]:
dots_x=range(1,30)
plt.plot(dots_x, r0)
plt.plot(dots_x, r1)
plt.plot(dots_x, r2)
plt.plot(dots_x, r3)
plt.plot(dots_x, r4)
plt.show()